## Initialize

In [ ]:
#library(Rmisc)
library(dtplyr)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)
library(data.table)
library("jsonlite")
library(ggthemes)

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

project_label="22_medical_records"
project_path = glue("{base_path}/results/projects/{project_label}")
figure_path = glue("{project_path}/figures")
output_path = glue("{project_path}/data")

experiment = 220627
experiment_path = glue("{output_path}/{experiment}")

In [ ]:
library(data.table)
library(arrow)

In [ ]:
endpoints_md = fread(glue("{experiment_path}/endpoints.csv"), colClasses=c("phecode"="character")) %>% as_tibble()
endpoints = sort(endpoints_md$endpoint)

In [ ]:
endpoints_md

In [ ]:
endpoint_defs = arrow::read_feather(glue("{output_path}/phecode_defs_220306.feather")) %>% arrange(endpoint)

In [ ]:
endpoint_selection = c(
    
   'phecode_401', #  "Hypertension", # intervention
    'phecode_202', #  "Diabetes mellitus", # intervention
    'phecode_416-21', #  "Atrial fibrillation", # intervention
    'phecode_468', #  "Pneumonia", # intervention
    'phecode_474', #  "Chronic obstructive pulmonary disease [COPD]", # interventio
    'phecode_583', #  "Chronic kidney disease", # intervention
    
    'phecode_404', #  "Ischemic heart disease",
    'phecode_404-1', #  "Myocardial infarction [Heart attack]", # intervention
    'phecode_431-11', #  "Cerebral infarction [Ischemic stroke]",
    'phecode_424', #  "Heart failure", # intervention
    'phecode_420', #  "Cardiac arrest", # intervention
    'OMOP_4306655', #  "All-Cause Death", # intervention
    
    'phecode_438-11',   #  "Abdominal aortic aneurysm",
    'phecode_440-3',#  "Pulmonary embolism", # intervention
    'phecode_413-21',#  "Aortic stenosis", # intervention
    'phecode_413-11', #  "Mitral valve insufficiency",
    'phecode_410-2',#  "Endocarditis",
    'phecode_400',#  "Rheumatic fever and chronic rheumatic heart diseases",	
    
    'phecode_164', #  "Anemia", # intervention
    'phecode_718',  #  "Back pain", # intervention
    'phecode_324-11', #  "Parkinson's disease (Primary)",
    'phecode_705-1', #  "Rheumatoid arthritis", # NEW + interventio
    'phecode_665', #  "Psoriasis", # interesting
    'phecode_284'#  "Suicide ideation and attempt or self harm" # intervention
)

endpoints_common = c(
    'phecode_401', #  "Hypertension", # intervention
    'phecode_202', #  "Diabetes mellitus", # intervention
    'phecode_416-21', #  "Atrial fibrillation", # intervention
    'phecode_468', #  "Pneumonia", # intervention
    'phecode_474', #  "Chronic obstructive pulmonary disease [COPD]", # interventio
    'phecode_583', #  "Chronic kidney disease", # intervention
    
    'phecode_404', #  "Ischemic heart disease",
    'phecode_404-1', #  "Myocardial infarction [Heart attack]", # intervention
    'phecode_431-11', #  "Cerebral infarction [Ischemic stroke]",
    'phecode_424', #  "Heart failure", # intervention
    'phecode_420', #  "Cardiac arrest", # intervention
    'OMOP_4306655' #  "All-Cause Death", # intervention)
    )
    
endpoints_cardio = c(
    'phecode_438-11',   #  "Abdominal aortic aneurysm",
    'phecode_440-3',#  "Pulmonary embolism", # intervention
    'phecode_413-21',#  "Aortic stenosis", # intervention
    'phecode_413-11', #  "Mitral valve insufficiency",
    'phecode_410-2',#  "Endocarditis",
    'phecode_400'#  "Rheumatic fever and chronic rheumatic heart diseases",	
)

endpoints_interesting = c(
    'phecode_164', #  "Anemia", # intervention
    'phecode_718',  #  "Back pain", # intervention
    'phecode_324-11', #  "Parkinson's disease (Primary)",
    'phecode_705-1', #  "Rheumatoid arthritis", # NEW + interventio
    'phecode_665', #  "Psoriasis", # interesting
    'phecode_284'#  "Suicide ideation and attempt or self harm" # intervention
)

endpoint_defs = endpoint_defs %>% 
    mutate(name = phecode_string) %>%
    mutate(name = 
           case_when( 
               phecode_string == "Myocardial infarction [Heart attack]"~"Myocardial infarction",
               phecode_string == "Cerebral infarction [Ischemic stroke]"~"Ischemic stroke",
               phecode_string == "Chronic obstructive pulmonary disease [COPD]"~"Chronic obstructive pulmonary disease",
               phecode_string == "Mitral valve insufficiency"~"Mitral insufficiency",
               phecode_string == "Parkinson's disease (Primary)"~"Parkinson's disease",
               phecode_string == "Suicide ideation and attempt or self harm"~"Suicide attempt",
               phecode_string == "Ischemic heart disease"~"Coronary heart disease",
               phecode_string == "Chronic kidney disease"~"Chronic kidney disease",
               phecode_string == "Rheumatic fever and chronic rheumatic heart diseases"~"Rheumatic heart disease",
               phecode_string == "Abdominal aortic aneurysm"~"Abdominal aortic aneurysm",
                  TRUE ~ name)
           )
            
endpoint_map = endpoint_defs$name
names(endpoint_map) =  endpoint_defs$endpoint
#endpoint_order = (endpoint_defs %>% arrange(as.numeric(phecode)))$endpoint
endpoint_order = endpoint_selection

In [ ]:
individualrisks = arrow::read_feather(glue("{experiment_path}/coxph/prediction_deltas_individualchange.feather"))

In [ ]:
individualrisks

In [ ]:
library(scales)

In [ ]:
add.alpha <- function(col, alpha=1){
  if(missing(col))
    stop("Please provide a vector of colours.")
  apply(sapply(col, col2rgb)/255, 2, 
                     function(x) 
                       rgb(x[1], x[2], x[3], alpha=alpha))  
}

In [ ]:
data_covariates = arrow::read_feather(glue("{output_path}/220627/data_covariates_full.feather"), col_select=c(eid, age, sex))

In [ ]:
plot_individual = function(eid){
    
    set.seed(125)
    library(circlize)
    library(wesanderson)

    options(repr.plot.width = 8, repr.plot.height = 8, repr.plot.res=320)

    #https://www.rdocumentation.org/packages/circlize/versions/0.4.15/topics/CELL_META

    endpoint_order = (endpoints_md)$endpoint

    categories_for_agg = c("Rx", "Preg", "Stat", "Death", "Signs/Symptoms")

    endpoints_md_subset = endpoints_md %>% mutate(category = case_when(phecode_category %in% categories_for_agg ~ "Other", TRUE ~phecode_category)) #%>% filter(freq>0.01)

    categories = (endpoints_md_subset %>% arrange(endpoint) %>% distinct(category))$category
    categories = c(categories %>% tail(length(categories)-1), categories %>% head(1), "Legend")

    df = individualrisks %>% filter(eid==!!eid) %>% 
        filter(endpoints %in% endpoints_md_subset$endpoint) %>%
        left_join(endpoints_md %>% select(endpoint, phecode, phecode_string, phecode_category, sex)) %>% 
        mutate(endpoint = factor(endpoint, levels=endpoint_order)) %>% ungroup() %>%
        mutate(category = case_when(phecode_category %in% categories_for_agg ~ "Other", TRUE ~phecode_category)) %>%
        mutate(category = factor(category, levels=categories)) %>% group_by(category) %>% 
        arrange(endpoint) %>% mutate(x = row_number(endpoint)) %>% ungroup() %>%
        mutate(prop_rel = case_when(prop_rel<1 ~ 1, TRUE ~prop_rel)) %>%
        mutate(rel_alpha = case_when(prop_rel<50 ~ 0.02*prop_rel, TRUE ~ 1)) %>% 
        mutate(abs_alpha = case_when(`Age+Sex+MedicalHistory`<0.5 ~ `Age+Sex+MedicalHistory`, TRUE ~ 1))  

    legend_x2 = round(200*nrow(endpoints_md_subset)/1883)

    sectors = df %>% select(category, x) %>% group_by(category) %>% 
        summarise(x1=min(x), x2=max(x)+1) %>% mutate(xcenter = round(x2/2)) %>% 
        mutate(gap.after=case_when(category=="Other"~1, TRUE ~ 1)) %>%
        add_row(category="Legend", x1=1, x2=legend_x2, xcenter=round(legend_x2/2), gap.after=1) %>%
        mutate(sep.col=case_when(category=="Legend"~"White", TRUE ~ add.alpha("black", 0.7))) %>%
        mutate(bg.col=case_when(category=="Legend"~"White", TRUE ~ add.alpha("black", 0.05))) %>%
        mutate(accent.col = add.alpha(c(wes_palette("Darjeeling1", 16, type = "continuous"), "White"), 0.5))
    
    circos.clear()

    circos.par("track.height"=0.2, cell.padding = c(0, 0, 0, 0), gap.degree = 1, track.margin=c(.00,.00), start.degree=74, clock.wise=T, gap.after=sectors$gap.after)

    circos.initialize(categories, xlim = sectors %>% select(x1, x2))

    df_selection = df %>% filter((prop_rel>2&`Age+Sex+MedicalHistory`>0.1)) #%>% filter(prop_rel>5&`Age+Sex+MedicalHistory`>0.01)  filter(`Age+Sex+MedicalHistory`>0.1)
    circos.labels(df_selection$category, x = df_selection$x, 
                  labels = str_wrap(glue("{df_selection$phecode_string} (x{round(df_selection$prop_rel, 1)}, {round(df_selection$`Age+Sex+MedicalHistory`*100, 1)}%)"), 20), 
                  side = "outside", cex=0.5, niceFacing=TRUE)

    circos.par("track.height"=0.01)
    circos.track(df$category, x = df$x, ylim=c(0, 1), bg.border = NA, bg.col=sectors$accent.col)

    # absolute risk estimates
    circos.par("track.height"=0.3)
    circos.track(df$category, x = df$x, y = df[["Age+Sex+MedicalHistory"]], ylim=c(0, 0.7), panel.fun = function(x, y) {

        # metadata
        sector.name = get.cell.meta.data("sector.index")
        xcenter = round(get.cell.meta.data("xlim")["max.data"]/2)

        # content
        alpha = (df %>% filter(category==!!sector.name))$rel_alpha
        circos.lines(x, y, lwd=2, col = add.alpha("black", alpha), type="h")
        circos.points(x, y, pch = 20, cex = 0.7, col = add.alpha("black", alpha))

        #style    
        if (sector.name!="Legend"){

            circos.text(xcenter, 0.65, sector.name, cex=0.55, facing="bending.inside", niceFacing=TRUE, col=add.alpha("black", 0.7))

            for (y in c(0, 0.1, 0.2, 0.3, 0.4, 0.5)){
                circos.lines(x, rep(y, length(x)), col=add.alpha("black", 0.1), pch = 20, cex = 0.5)
            }
        }


        if (sector.name=="Legend"){
            legend_sector = sectors %>% filter(category=="Legend")
            xs=0:legend_sector$x2

            xcenter = round(get.cell.meta.data("xlim")["max.data"]/2)
            circos.text(xcenter, 0.65, "10-year risk", cex=0.6, facing="bending.inside", niceFacing=TRUE, col=add.alpha("black", 0.7))

            for (y in c(0, 0.1, 0.2, 0.3, 0.4, 0.5)){
                circos.lines(xs, rep(y, length(xs)), col=add.alpha("black", 0.1), pch = 20, cex = 0.5)
                circos.text(xcenter, y+0.04, glue("{round(y*100)} %"), cex=0.4, facing="bending.inside", niceFacing=TRUE, col=add.alpha("black", 0.7))
                }
        }


    }, bg.border = NA, bg.col = sectors$bg.col)

    # separation segment
    set_track_gap(mm_h(1)) # 2mm

    # predisposition
    circos.par("track.height"=0.3)
    circos.track(df$category, x = df$x, y = log(df[["prop_rel"]]), ylim=c(0, log(20000)), panel.fun = function(x, y) {

        # metadata
        sector.name = get.cell.meta.data("sector.index")
        xcenter = round(get.cell.meta.data("xlim")["max.data"]/2)

        # content
        alpha = (df %>% filter(category==!!sector.name))$rel_alpha
        circos.lines(x, y, lwd=2, col = add.alpha("black", alpha), type="h")
        circos.points(x, y, pch = 20, cex = 0.7, col = add.alpha("black", alpha))


        # style   
        if (sector.name!="Legend"){
            for (y in c(log(1), log(10), log(100), log(1000))){
                circos.lines(x, rep(y, length(x)), col=add.alpha("black", 0.1), pch = 20, cex = 0.5)
                }
            }
        else{
            circos.text(xcenter, log(8000), "Predisposition", cex=0.6, facing="bending.inside", niceFacing=TRUE, col=add.alpha("black", 0.7))
            for (y in c(log(1), log(10), log(100), log(1000))){
                xs=0:(sectors %>% filter(category=="Legend"))$x2
                circos.lines(xs, rep(y, length(xs)), col=add.alpha("black", 0.1), pch = 20, cex = 0.5)

                if (y==log(1)){label="x1"}
                if (y>log(1)){label=glue("x{round(exp(y))}")}

                circos.text(xcenter, log(exp(y)*2), label, cex=0.4, facing="bending.inside", niceFacing=TRUE, col=add.alpha("black", 0.7))
                }
            }


    }, bg.border = NA, bg.col = sectors$bg.col)
}

In [ ]:
# get medical history for the record

# main figure
#eid = XXXXX # main figure #XXXXX #XXXXX #ischemic heart disease

### Supplement 2 - actionable and preventable - stop development
#eid = XXXXX # all-cause death
#eid = XXXXX # suicide

### Supplement 3 - actionable and treatable -> slower progress
#eid = XXXXX # aortic stenosis and mitral insufficiencies
eid = XXXXX # anemia



# Figure 4 - actionable cardiovascular endpoints
#eid = XXXXX # ischemic heart disease


### actionable and treatable -> slower progress
# eid = XXXXX # hepatitis
#eid = XXXXX #  sepsis
 #eid = XXXXX #XXXXX # heart failure
# eid = XXXXX # backpain
# eid = XXXXX # COPD
# eid = XXXXX # Psoriasis
# eid = XXXXX # rheumatoid arthritis
#eid = XXXXX # anemia
#eid = XXXXX # back pain

### not medically actionable, no treatment but otherwise 

print(data_covariates %>% select(eid, age, sex) %>% filter(eid == !!eid))
plot_individual(eid)

# eid = XXXXX # individual is very sick	# cardiac arrest # 

In [ ]:
eid = XXXXX
pdf(glue("outputs/Figure3d_Individual_{eid}.pdf"), width=8, height=8)
plot_individual(eid)
dev.off()

In [ ]:
eid = XXXXX
pdf(glue("outputs/SupplFigure2a_Individual_{eid}.pdf"), width=8, height=8)
plot_individual(eid)
dev.off()

In [ ]:
eid = XXXXX
pdf(glue("outputs/SupplFigure2b_Individual_{eid}.pdf"), width=8, height=8)
plot_individual(eid)
dev.off()

In [ ]:
eid = XXXXX
pdf(glue("outputs/SupplFigure3a_Individual_{eid}.pdf"), width=8, height=8)
plot_individual(eid)
dev.off()

In [ ]:
eid = XXXXX
pdf(glue("outputs/SupplFigure3b_Individual_{eid}.pdf"), width=8, height=8)
plot_individual(eid)
dev.off()